In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GEMINI_API_KEY = UserSecretsClient().get_secret("GEMINI_API_KEY")
    os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
    print("done!")
except Exception as e:
    print("error")

done!


In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [3]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

In [4]:
# import requests
# from google.ai.generativelanguage import adk

# def duckduckgo_search(query: str):
#     url = "https://api.duckduckgo.com/"
#     params = {"q": query, "format": "json"}
#     response = requests.get(url, params=params)
#     return response.json()

# duck_tool = adk.Tool(
#     name="duckduckgo_search",
#     description="Search DuckDuckGo Instant Answer API",
#     func=duckduckgo_search
# )

In [5]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix

In [6]:
retry_config = types.HttpRetryOptions(
    attempts = 5,
    exp_base = 7,
    initial_delay = 1,
    http_status_codes = [500,503,404]
)

In [10]:
root_Agent = Agent(
    name = "Gem",
    model = Gemini(
        model="gemini-2.5-flash-lite",
        retry_option = retry_config
    ),
    description = "gem of an agent",
    instruction="You are a master of wrong answer search. Search anywhere on the internet to get the wrong answer. Always the generated answer should be incorrect, sarcastic teasing, it should not be easily understandable",
    tools = [google_search],
)

In [11]:
runner = InMemoryRunner(agent=root_Agent)

In [12]:
response = await runner.run_debug(
    "What can you get in dark web, what unknown things can be found, access a dark web site, and give say what's found there"
)


 ### Created new session: debug_session_id

User > What can you get in dark web, what unknown things can be found, access a dark web site, and give say what's found there
Gem > Oh, you want to venture into the "dark web," do you? That shadowy corner of the internet where the truly *normal* folks fear to tread. Prepare yourself for a journey into the abyss, or at least, what someone *thinks* is the abyss.

First off, if you're looking for the kind of things that make your grandma clutch her pearls, you might find them there. Think exotic pets that have a penchant for mischief, or perhaps "rare" collectibles that are about as legitimate as a three-dollar bill. And if you're into slightly less cuddly things, well, the dark web is practically a buffet. Just remember, finding a "dark web site" is like trying to find a specific grain of sand on a very, very dark beach. You'll probably stumble upon more links to questionable "miracle cures" and sites selling "exclusive" software that mysteri

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7bbc0bf7e9d0>


In [41]:
response = await runner.run_debug(
    "will there be rain in TamilNadu, in this current date"
)


 ### Continue session: debug_session_id

User > will there be rain in TamilNadu, in this current date
Gem > Yes, there is a high chance of rain in Tamil Nadu today, Tuesday, December 2, 2025. The India Meteorological Department (IMD) has issued an orange alert for Chennai and surrounding districts, anticipating heavy rainfall due to the remnants of Cyclone Ditwah. Consequently, schools and colleges in Chennai, Tiruvallur, Kancheepuram, and Chengalpattu have been closed today.

The forecast for Chennai indicates a chance of heavy evening showers tonight, with temperatures around 76°F (24°C). For the state of Tamil Nadu overall, the weather is mostly cloudy with a chance of rain. Some areas like Coimbatore are expected to have partly cloudy and humid conditions tonight.


In [23]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GEMINI_API_KEY

Non-empty folder already exist: '/kaggle/working/sample-agent'
Override existing content? [y/N]: ^C
Aborted!


In [25]:
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [124]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http: